This notebook serves as an example to analyze and display muon parameters created from a simtel file and written to an hdf5 table using the MuonParameterCalculation notebook.

Created by Markus Gaug, 19/3/2020, based on code found in ctapipe.tools.MuonDisplayerTool and ctapipe.image.muon.*



Import the table, created by MuonParameterCalculation

In [ ]:
import tables as tb
import glob
import numpy as np
from astropy import units as u
from os.path import basename

muonFile="/muon_30deg_84.6805deg_run2307___cta-prod4-sst-astri_desert-2180m-LaPalma-sst-astri.simtel.hdf5"
#muonFile="/muon_0deg_354.681deg_run53___cta-prod4-sst-astri_desert-2180m-LaPalma-sst-astri.simtel.hdf5"
#muonFile="/muon_30deg_84.6805deg_run925___cta-prod4-lst_desert-2180m-LaPalma-lst.simtel.hdf5"
muonDir="/Users/markusgaug/Teaching/JordiFalguera/Muons"
muonTableName = 'muons/muon_table'


'''Initialize all parameters we might need in this notebook from the hdf5 table 
   I'm really sorry, no way found to combine tables which host data of type 'Table'
   So, we have to read all files in once and store all the information we need in separate np arrays'''

usedParameters = [  'alt' , 'az', 
            'center_x', 'center_y',  'radius', 
            'impact_x', 'impact_y', 
            'core_x', 'core_y',
            'energy',
            'optical_efficiency', 
            'width',
            'impact_error', 'phi_error', 'width_error', 'optical_efficiency_error',
         ]

telParameters = [ 'telalt', 'telaz']

'''and the corresponding units'''
usedUnits = { 'alt' : u.deg, 'az': u.deg, 
              'center_x': u.deg, 'center_y': u.deg, 'radius' : u.deg, 
              'impact_x' : u.m, 'impact_y' : u.m, 
              'core_x' : u.m, 'core_y' : u.m,
              'energy' : u.TeV, 
              'optical_efficiency' : u.dimensionless_unscaled,
              'width' : u.deg, 
              'impact_error' : u.m, 'phi_error' : u.deg, 'width_error' : u.deg, 
              'optical_efficiency_error' : u.dimensionless_unscaled
            }

telUnits = { 'telalt' : u.deg , 'telaz' : u.deg }

'''Initialize the dictionary for easier access later and loops '''
muonData = {}
for parameterName in usedParameters:
    muonData[parameterName] = np.array([]) 
for telparName in telParameters:
    muonData[telparName] = np.array([]) 
#print (muonData)

if '*' in muonFile:
    fileCounter = 0
    print ('Search for files in',muonDir+muonFile)
    for file in glob.glob(muonDir+muonFile):
        print ('reading ',file)
        tel_pointing_alt = 90. - float(((basename(file).split('_')[1]).split('d'))[0])
        tel_pointing_az = float(((muonFile.split('_')[2]).split('d'))[0])
        telData = { 'telalt' : tel_pointing_alt, 'telaz' : tel_pointing_az }
        h5Root  = tb.open_file(file,mode='r')  
        h5Table = h5Root.root[muonTableName]
        #print (h5Table)
        for x in h5Table.iterrows(): print (x)
        for parameterName in usedParameters:
            arrayWithUnit = ( np.array(  [ x[parameterName] for x in h5Table.iterrows() ] ) ) * usedUnits[parameterName]
            muonData[parameterName] = np.concatenate( (muonData[parameterName], arrayWithUnit) )
            #print (parameterName,muonData[parameterName])
        for telparName in telParameters: 
            #print (len(h5Table), telData[telparName])
            arrayWithUnit = ( np.ones(len(h5Table))  * telData[telparName] ) #* telUnits[telparName]
            muonData[telparName] = np.concatenate( (muonData[telparName], arrayWithUnit) )
        h5Root.close()
        fileCounter = fileCounter + 1
    print('\nDone with reading all {:d} muon data arrays from {:d} df5 files into memory'.format(len(usedParameters)
                                                                                              ,fileCounter))
else:
    file = muonDir+muonFile
    print ('reading ',file)
    tel_pointing_alt = 90. - float(((muonFile.split('_')[1]).split('d'))[0])
    tel_pointing_az = float(((muonFile.split('_')[2]).split('d'))[0])
    telData = { 'telalt' : tel_pointing_alt, 'telaz' : tel_pointing_az }
    print ('Telescope points at: ', tel_pointing_alt,'deg Alt and ', tel_pointing_az,'deg Az ')
    h5Root  = tb.open_file(file)
    print (h5Root)
    print (h5Root.root.muons.muon_table)
    h5Table = h5Root.root.muons.muon_table
    print (h5Table.flush)
    for x in h5Table.itercolumns(): print (x)
    for parameterName in usedParameters:
        arrayWithUnit = ( np.array(  [ x[parameterName] for x in h5Table.iterrows() ] ) ) * usedUnits[parameterName]
        #print (len(arrayWithUnit))
        muonData[parameterName] = np.concatenate( (muonData[parameterName], arrayWithUnit) )
    for telparName in telParameters: 
        #print (len(h5Table), telData[telparName])
        arrayWithUnit = ( np.ones(len(h5Table))  * telData[telparName] ) #* telUnits[telparName]
        muonData[telparName] = np.concatenate( (muonData[telparName], arrayWithUnit) )
    #print (h5Table.iterrows())
    print('\nDone with reading all {:d} muon data arrays from the hdf5 file into memory'.format(len(usedParameters)))

First, check the accuracy of the ring reconstruction.  

In [ ]:

from astropy.coordinates import SkyCoord, AltAz
from ctapipe.coordinates import NominalFrame


The muon reconstruction found in ctapipe.image.muon.muon_reco_functions converts camera coordinates immediately into sky coordinates and reconstructs the ring 'on the sky'. So, we only have to compare the true muon direction with the one found in 'ring_center_x' and 'ring_center_y', which are actually 'ring_center_delta_az' and 'ring_center_delta_alt'. 

However, it looks as if the camera rotation (particularly important for FlashCam) has been taken into account, so have to subtract it from the true muon incidence angle. Moreover, the camera rotation angle cannot be written to the hdf5 table, because event.inst.subarray.tels[1].camera does not have any Fields. The should actually be fixed... 

We will have to initialize the rotation angle by hand

In [ ]:
"""These values apply only to FlashCam !!!"""
camera_rotation = 0.* u.deg # rotation seems to be correced with current version, previously 30.*u.deg
"""For Nectar Cam and LST Cam use:
camera_rotation = 100.893 * u.deg """


Initialize the telescope pointing direction now. The values can also be found later on in max_alt/min_alt or in max_az/min_az, but for simplicity we copy them here and initialize only once.

Moreover, be aware with using the value from max_alt/min_alt directly (1.5707964 rad, beware of the last digit '4'), because AltAz will reject them telling that an altitude larger than 90 deg. is obtained and not supported. Reducing the altitude to 1.5707963 rad by hand works, but should actually be fixed in the simtel_array. 

In [ ]:
telescope_alt =  u.Quantity(muonData['telalt'],u.deg).to(u.rad) #1.5707963  * u.rad   # If muons have been simulated with telescope pointing to zenith 
telescope_az  =  u.Quantity(muonData['telaz'],u.deg).to(u.rad) #- 0.09284279 * u.rad   # Magnetic North, used for simulations with Corsika 

""" The next lines are only to avoid warnings from ctapipe.coordinates and are not needed for MC """
from astropy.time import Time
from astropy.coordinates import EarthLocation
""" Adopt a dummy time and an earth location"""
obstime = Time('2013-11-01T03:00')
location = EarthLocation.of_site('Roque de los Muchachos')
altaz = AltAz(location=location, obstime=obstime)

telescope_pointing = SkyCoord(alt=telescope_alt, az=telescope_az, frame=altaz)

Now, get the true muon directions into the right frame: 

In [ ]:
alts = muonData['alt']
"""
ATTENTION !!!! 
HAVE TO SUBTRACT HERE THE CAMERA ROTATION FOR FLASHCAM WHICH WAS NOT CORRECTLY 
TAKEN INTO ACCOUNT DURING THE PRODUCTION OF THE MUON PARAMETERS !!!!
"""
azs  = muonData['az'] - camera_rotation

#print (alts)

"""Convert the original muon directions into the telescope pointing frame """
ring_original       = SkyCoord(az=azs, alt=alts, frame=altaz)
ring_original_sky   = ring_original.transform_to( NominalFrame(origin=telescope_pointing) )

ring_original_sky_x = ring_original_sky.fov_lon.to(u.deg)
ring_original_sky_y = ring_original_sky.fov_lat.to(u.deg)

#print (ring_original_sky_x)
#print (ring_original_sky_y)

The reconstructed ring already comes in sky coordinates

In [ ]:
ring_delta_az  = muonData['center_x']
ring_delta_alt = muonData['center_y']
radius         = muonData['radius']

Now, check the angular distance between both and plot them as a function of energy

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
from astropy.stats import sigma_clip, mad_std
from scipy.optimize import minimize
from scipy.optimize import least_squares

d_x = ring_original_sky_x - ring_delta_az
d_y = ring_original_sky_y - ring_delta_alt
d_r = np.sqrt(d_x**2 + d_y**2)  

energy = muonData['energy']

plt.figure(figsize=[10.,5.])
plt.plot(np.log10(energy.to_value()),d_r.to_value(u.deg),'.') 
plt.xlabel(r'$\log_{10}$(energy [TeV])')
plt.ylabel(r'$\Delta \Phi$ [deg]')
plt.ylim([0,1.0])


# Fit with robust statistics

def deviationError(x,a,b,c):
    return a*np.abs(x)**c + b

def residuals(x,t,y):
    return deviationError(t,x[0],x[1],x[2]) - y

bnds = ((0, None), (0, None), (0, None))
f_scale = 0.01
loss = 'soft_l1'
result = least_squares(residuals, [0.02, 0.2, 3.], args=(np.log10(energy.to_value()),d_r.to_value(u.deg)),loss=loss, f_scale=f_scale) #, bounds=bnds)

xx = np.linspace(-2.2,-0.01,100)
label_str = rf'$\Delta \Phi = {result.x[1]:3.3f} + {result.x[0]:3.3f} \cdot | \log_{{{10}}}(energy [TeV]) |^{{{result.x[2]:3.1f}}}$'
plt.plot(xx, deviationError(xx, *result.x), label=label_str) 
plt.legend(loc='best')


plt.show()


Let's check also the accuracy of the reconstructed muon impact distance. 

Here, also a rotation the true muon impact parameters is necessary and a reflection of the X-coordinate! 

First, try without anything: 

In [ ]:
reconstructed_core_x = muonData['impact_x']
reconstructed_core_y = muonData['impact_y']
reconstructed_core_d = np.sqrt(reconstructed_core_x**2+reconstructed_core_y**2)

true_core_x = muonData['core_x']
true_core_y = muonData['core_y']
true_core_d = np.sqrt(true_core_x**2+true_core_y**2)

reconstruction_err = np.sqrt((reconstructed_core_x-true_core_x)**2+(reconstructed_core_y-true_core_y)**2)
reconstruction_err_d = reconstructed_core_d - true_core_d
unit = reconstruction_err.unit

plt.figure(figsize=[15.,5.])
plt.subplot(121)
plt.hist(reconstruction_err.to_value())
plt.xlabel(rf'$\Delta \rho$ [{unit:FITS}]')
plt.subplot(122)
plt.hist(reconstruction_err_d.to_value())
plt.xlabel(rf'$\Delta |\rho|$ [{unit:FITS}]')
plt.show()


Now, try a reflection only:

In [ ]:
reconstruction_err = np.sqrt((reconstructed_core_x+true_core_x)**2+(reconstructed_core_y-true_core_y)**2)

plt.figure(figsize=[15.,5.])
plt.subplot(121)
plt.hist(reconstruction_err.to_value())
plt.xlabel(rf'$\Delta \rho$ [{unit:FITS}]')
plt.subplot(122)
plt.hist(reconstruction_err_d.to_value())
plt.xlabel(rf'$\Delta |\rho|$ [{unit:FITS}]')
plt.show()


Seems that here also, the camera rotation needs to be introduced by hand:

In [ ]:
camera_rotation = 84.* u.deg #   79.* u.deg  # 260 ++
theta = camera_rotation.to_value('rad')
rotM = np.array([[np.cos(theta), -np.sin(theta)], 
                 [np.sin(theta),  np.cos(theta)]])

rotated_cores = np.dot(rotM, np.stack((true_core_x,true_core_y)))

In [ ]:
# first some basic quality cuts 
ids = np.where((reconstructed_core_d > 1 * u.m) & (reconstructed_core_d < 12 * u.m) & (radius < 1.3 *u.deg) & (radius > 0.85 *u.deg))

reconstruction_err = np.sqrt((reconstructed_core_x-rotated_cores[0])**2+(reconstructed_core_y-rotated_cores[1])**2)
true_core_d = np.sqrt(rotated_cores[0]**2+rotated_cores[1]**2)
reconstruction_err_d = reconstructed_core_d - true_core_d


mean   = reconstruction_err[ids].to_value().mean()
mean_d = reconstruction_err_d[ids].to_value().mean()
std_d  = reconstruction_err_d[ids].to_value().std()

plt.figure(figsize=[12.,12.])
plt.subplot(221)
plt.hist(reconstruction_err[ids].to_value(),30)
plt.axvline(mean, color='k', linestyle='dashed', linewidth=1, label=f'Mean: {mean:.4f} {unit:FITS} ')
plt.xlabel(rf'$|\Delta \rho|$ [{unit:FITS}]')
plt.legend()

plt.subplot(222)
plt.hist(reconstruction_err_d[ids].to_value(),30)
plt.axvline(mean_d, color='k', linestyle='dashed', linewidth=1, label=rf'$\Delta|\rho|$: {mean_d:.2f} $\pm$ {std_d:.2f} {unit:FITS} ')
plt.xlabel(r'$|\rho|_{{reconstructed}}-|\rho|_{{simulated}}$ [{0:FITS}]'.format(unit))
plt.legend()

plt.subplot(223)
plt.plot(true_core_d[ids].to_value(),reconstruction_err[ids].to_value(),'.')
plt.xlabel(r'$|\rho_{{simulated}}|$ [{0:FITS}]'.format(unit))
plt.ylabel(r'$|\Delta \rho|$ [{0:FITS}]'.format(unit))


plt.subplot(224)
plt.plot(true_core_d[ids].to_value(),reconstruction_err_d[ids].to_value(),'.')
plt.xlabel(r'$|\rho_{{simulated}}|$ [{0:FITS}]'.format(unit))
plt.ylabel(r'$|\rho|_{{reconstructed}}-|\rho|_{{simulated}}$ [{0:FITS}]'.format(unit))

plt.show()

That seems to be correct one, although still quite improvable in precision! 

Let's check whether it has to do with muon energy: 

In [ ]:
plt.plot(np.log10(energy[ids].to_value()),reconstruction_err[ids].to_value(),'.')
plt.xlabel(r'$\log_{10}$(energy [TeV])')
plt.ylabel(r'$\Delta \rho$ [{0:FITS}]'.format(unit))

result = least_squares(residuals, [0.02, 0.2, 3.], args=(np.log10(energy[ids].to_value()),reconstruction_err[ids].to_value()),loss=loss, f_scale=f_scale) #, bounds=bnds)

xx = np.linspace(-2.2,-0.01,100)
label_str = rf'$\Delta \rho = {result.x[1]:3.3f} + {result.x[0]:3.3f} \cdot | \log_{{{10}}}(energy [TeV]) |^{{{result.x[2]:3.1f}}}$'
plt.plot(xx, deviationError(xx, *result.x), label=label_str) 
plt.legend(loc='best')

plt.show()

Definitely! We have to cut also on the muon ring! 

In [ ]:
thetainf = 1.225 * u.deg    # average value of sqrt(2epsilon) for 2200 m a.s.l.
m_mu     = 105.7 * u.MeV 

radius_predicted = np.sqrt(thetainf.to('', equivalencies=u.dimensionless_angles())**2 
                           - (m_mu.to(u.TeV)/energy)**2).to(u.deg,equivalencies=u.dimensionless_angles())

plt.figure(figsize=[12.,5.])
plt.subplot(121)
plt.plot(np.log10(energy[ids].to_value()),radius[ids].to_value(),'.')
plt.xlabel(r'$\log_{10}$(energy [TeV])')
plt.ylabel(r'$R_{{reconstructed}}$ [deg]')

plt.subplot(122)
plt.plot(radius_predicted[ids].to_value(),radius[ids].to_value(),'.')
plt.plot([0.9,1.3],[0.9,1.3])
plt.xlim(0.9,1.28), plt.ylim(0.9,1.28)

plt.xlabel(r'$R_{{simulated}}$ [deg]')
plt.ylabel(r'$R_{{reconstructed}}$ [deg]')

plt.show()

In [ ]:
# new quality cuts, with a stronger ring radius cut  
ids = np.where((reconstructed_core_d > 1 * u.m) & (reconstructed_core_d < 12 * u.m) 
               & (radius < 1.3 *u.deg) & (radius > 1.17 *u.deg))

mean   = reconstruction_err[ids].to_value().mean()
mean_d = reconstruction_err_d[ids].to_value().mean()
std_d  = reconstruction_err_d[ids].to_value().std()

plt.figure(figsize=[12.,12.])
plt.subplot(221)
plt.hist(reconstruction_err[ids].to_value(),30)
plt.axvline(mean, color='k', linestyle='dashed', linewidth=1, 
            label=f'Mean: {mean:.4f} {unit:FITS} ')
plt.xlabel(rf'$|\Delta \rho|$ [{unit:FITS}]')
plt.legend()

plt.subplot(222)
plt.hist(reconstruction_err_d[ids].to_value(),30)
plt.axvline(mean_d, color='k', linestyle='dashed', linewidth=1, 
            label=rf'$\Delta|\rho|$: {mean_d:.2f} $\pm$ {std_d:.2f} {unit:FITS} ')
plt.xlabel(r'$|\rho|_{{reconstructed}}-|\rho|_{{simulated}}$ [{0:FITS}]'.format(unit))
plt.legend()

plt.subplot(223)
plt.plot(true_core_d[ids].to_value(),reconstruction_err[ids].to_value(),'.')
plt.xlabel(r'$|\rho_{{simulated}}|$ [{0:FITS}]'.format(unit))
plt.ylabel(r'$|\Delta \rho|$ [{0:FITS}]'.format(unit))

plt.subplot(224)
plt.plot(true_core_d[ids].to_value(),reconstruction_err_d[ids].to_value(),'.')
plt.xlabel(r'$|\rho_{{simulated}}|$ [{0:FITS}]'.format(unit))
plt.ylabel(r'$|\rho|_{{reconstructed}}-|\rho|_{{simulated}}$ [{0:FITS}]'.format(unit))

plt.show()

Finally, look at the optical efficienies: 

In [ ]:
effs = muonData['optical_efficiency']

plt.plot(np.log10(energy.to_value()),effs.to_value(),'.')
plt.xlabel(r'$\log_{10}$(energy [TeV])')
plt.ylabel(r'$\epsilon$ [1]')
plt.show()

Fair enough, let's check the values above 20 GeV only

In [ ]:
plt.plot(reconstruction_err[ids].to_value(),effs[ids].to_value(),'.')
plt.xlabel(r'$\Delta \rho$ [m]')
plt.ylabel(r'$\epsilon$ [1]')
plt.show()

In [ ]:
from scipy.stats import norm
from astropy.stats import biweight
from astropy.stats import mad_std

ids2 = np.intersect1d(ids,np.where((effs < 0.14) & (effs > 0.09)))

n, bins, patches = plt.hist(effs[ids2].to_value(),50, density=1, facecolor='green', alpha=0.75)
(mu, sigma) = norm.fit(effs[ids2].to_value())

def gauss(x,mu,sigma,A):
    return (A/np.sqrt(2*np.pi*sigma**2))*np.exp(-0.5*(((x-mu)/sigma))**2)

print ('mean, std dev. (full sample): ',effs[ids].mean(),effs[ids].std())
print ('median, MAD: ',np.median(effs[ids]),mad_std(effs[ids]))
print ('biweight loc, scale: ',biweight.biweight_location(effs[ids]),biweight.biweight_scale(effs[ids]))
print ('gaussian fit to reduced sample: ',mu,sigma)
l = plt.plot(bins, gauss(bins,mu,sigma,1.), 'r--', linewidth=2)

plt.xlabel(r'$\epsilon$ [1]')
plt.show()